# Netflix Investigation

In [ ]:
!pip install pymysql

In [2]:
import sqlalchemy as sal
import pandas as pd
from dotenv import load_dotenv
import os

In [4]:
load_dotenv(dotenv_path='../Keys/keys.env')

# Construct connection string based on database type
db_type = os.getenv('DB_TYPE').lower()

conn_string = (
    f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}"
    f"/{os.getenv('DB_NAME')}"
)

# Create database engine
engine = sal.create_engine(conn_string)
conn = engine.connect()


In [6]:
df = pd.read_csv('../netflix_titles.csv')

In [16]:
try:
    df.to_sql(
        name='netflix_titles',  # table name in your database
        con=engine,              # SQLAlchemy engine
        if_exists='append',     # 'replace' to drop & recreate table, 'append' to add to it
        index=False              # don't write pandas DataFrame index as a column
    )
    print("✅ Data written to database successfully!")

except Exception as e:
    print("❌ Error writing DataFrame to database:")
    print(e)


✅ Data written to database successfully!


#### This recreates the table to have the correct sized columns and to handle foreign characters.

